In [1]:
import numpy as np
import scipy.linalg as sal

In [2]:
def inner_product(left_vec, right_vec):
    bra = (np.conj(left_vec)).T
    ket = right_vec
    inner_prod = bra @ ket
    return inner_prod

In [3]:
def norm_column_vec(x):
    # m, n = x.shape
    inner_pro = inner_product(x, x)
    # inner_prodcut is still a 1x1 array, lets extract the element and take its sqrt
    norm = np.sqrt(np.squeeze(inner_pro))
    x_norm = x / norm
    return x_norm

In [5]:
def individual_spin_operation_matrix():
    S_1_2 = np.array([[3/4,0,0,0],[0,3/4,0,0],[0,0,3/4,0], [0,0,0,3/4]])
    S_1z = np.array([[1/2,0,0,0],[0,1/2,0,0],[0,0,-1/2,0], [0,0,0,-1/2]])
    S_2z = np.array([[1/2,0,0,0],[0,-1/2,0,0],[0,0,1/2,0], [0,0,0,-1/2]])
    S_1_plus = np.array([[0,0,1,0],[0,0,0,1],[0,0,0,0], [0,0,0,0]])
    S_2_plus = np.array([[0,1,0,0],[0,0,0,0],[0,0,0,1], [0,0,0,0]])
    S_1_minus = np.array([[0,0,0,0],[0,0,0,0],[1,0,0,0], [0,1,0,0]])
    S_2_minus = np.array([[0,0,0,0],[1,0,0,0],[0,0,0,0], [0,0,1,0]])
    
    operate = {"S_1_2": S_1_2,
               "S_2_2": S_1_2,
               "S_1z": S_1z,
               "S_2z" : S_2z,
               "S_1_plus": S_1_plus,
               "S_2_plus" : S_2_plus,
               "S_1_minus": S_1_minus,
               "S_2_minus" : S_2_minus
              }
    return operate

In [6]:
def S_2_operate(function_list):
    l = len(function_list)
    mat = np.zeros((l,l))
    operate = individual_spin_operation_matrix()
    S_1_2 = operate["S_1_2"]
    S_2_2 = operate["S_2_2"]
    S_1_plus = operate["S_1_plus"]
    S_2_plus = operate["S_2_plus"]
    S_1_minus = operate["S_1_minus"]
    S_2_minus = operate["S_2_minus"]
    S_1z = operate["S_1z"]
    S_2z = operate["S_2z"]
    
    S_2_operation = S_1_2 + S_2_2 + 2*(0.5*(S_1_plus@S_2_minus+ S_1_minus@S_2_plus) + S_1z@S_2z)
    
    for i in range(l):
        for j in range(l):
            mat[i,j] = np.squeeze(function_list[i].T @ S_2_operation @ function_list[j])
            
    return mat

In [4]:
def product_bases(eig_vec):
    up_up = eig_vec[0].reshape(-1,1)
    up_down = eig_vec[1].reshape(-1,1)
    down_up = eig_vec[2].reshape(-1,1)
    down_down = eig_vec[3].reshape(-1,1)
    
    function_list = [up_up, up_down, down_up, down_down]
    function_dict = {"up_up": up_up,
                "up_down": up_down,
                "down_up": down_up,
                "down_down": down_down}
    return function_list, function_dict

In [7]:
operate = individual_spin_operation_matrix()
S_1_2 = operate["S_1_2"]
val, vec = sal.eig(S_1_2)
function_list, function_dict = product_bases(vec)

In [10]:
S_2_mat = S_2_operate(function_list)
coupled_val ,coupled_vec = sal.eig(S_2_mat)
CG_matrix = coupled_vec.T

In [11]:
t1 = CG_matrix[2]
t0 = CG_matrix[0]
t_1 = CG_matrix[3]
s0 = CG_matrix[1]
coupled_states = {"triplet1": t1,
                  "triplet0": t0,
                  "triplet_1": t_1,
                  "singlet": s0
                 }


                                ## Interaction Hamiltonian

### H = lambda* (S1@S2)
since, Hamiltonian here depends on all three compoents of S1 and S2;
it will not commute with either S1z and S2z;
hence, states of definite m1s, m2s, such as |alpha,alpha>, |beta, beta> can not be eigenfucntion of H. On the other hand, H does commutes with S^2 (the effective spin operator of whole system), as we can write "S1 @ S2" in terms of already known system operator(S^2) and individual spin operators(S1^2, S2^2);
#### H = (lambda/2)* (S^2 - S1^2 - S2^2)

In [12]:
def interaction_hamiltonian(lambda_, S_2):
    indi_spin_mat = individual_spin_operation_matrix()
    S_1_2 = indi_spin_mat["S_1_2"]
    S_2_2 = indi_spin_mat["S_2_2"]
    H = (lambda_/2) * (S_2 - S_1_2 - S_2_2)
    
    return H

In [22]:
def interaction_energy_cal(left, right, lambda_, S_2):
    left_vec = norm_column_vec(left)
    right_vec = norm_column_vec(right)
    H = interaction_hamiltonian(lambda_, S_2)
    
    e = left_vec.T @ H @ right_vec
    
    return e

In [29]:
lambda_ = 0.5
bra_vec = coupled_states["singlet"]
ket_vec = coupled_states["singlet"]
e = interaction_energy_cal(bra_vec, ket_vec, lambda_, S_2_mat)
e

-0.37500000000000006